# Demo 3: Glue

## Other formats

Because you can create a pandas dataframe from many types of formats, you can use the same munging code regardless of data source.
https://pandas.pydata.org/pandas-docs/stable/io.html

![Pandas IO](assets/pandas_io.png)

### Get USAspending data via API

In [ ]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

uri = 'https://api.usaspending.gov/api/v2/search/spending_by_award/'
headers = {'content-type': 'application/json'}
awards_json = []
page = 1
while page:
    payload = {
        "filters":{"time_period":[{"start_date":"2009-10-01","end_date":"2018-09-30"}],
        "award_type_codes":["02","03","04","05"],
        "place_of_performance_locations":[{"country":"USA","state":"MA","county":"015"}]}, # Hampshire County
        "fields":["Award ID","Recipient Name","Start Date","End Date","Award Amount","Awarding Agency","Awarding Sub Agency","Award Type", "Description"],
        "page":page,
        "limit":100
    }
    r = requests.post(uri, data=json.dumps(payload), headers=headers)  
    awards_json.extend(r.json()['results'])
    if r.json()['page_metadata']['hasNext']:
        page = page + 1
    else:
        page = None

awards_df = pd.DataFrame(json_normalize(awards_json))
awards_df[:10]